In [1]:
import random
import math
import timeit
from numpy import unique
from numpy import array
from numpy import mean
from numpy import std
from numpy import bincount
from numpy import nonzero
import itertools
import copy
import time
import csv
import pandas as pd
import ast

In [2]:
class Card():
    def __init__(self,value,suit):
        self.value = value
        self.suit = suit
    def show(self):
        print(self.value,"of",self.suit)

In [3]:
class Deck():
    def __init__(self):
        self.cards = []
        suits = ["Hearts","Clubs","Diamonds","Spades"]
        values = range(2,15)
        for val in values:
            for suit in suits:
                self.cards.append(Card(val,suit))
        self.shuffle() #Manually shuffle new deck
    def shuffle(self):
        random.shuffle(self.cards)
    def show(self):
        for card in self.cards:
            card.show()
    def draw(self, pos = 0):
        ret = self.cards.pop(pos)
        return ret
    def bottom(self,card):
        self.cards.append(card)
    def showTop(self,num=5):
        index = 0
        while index < num:
            self.cards[index].show()
            index+=1

In [4]:
class PlayerHand():
    #Initial call merely initializes an empty hand
    #User must then dictate the kind of hand to be given
    def __init__(self):
        self.hand = [] #Will be list of card objects
    def show(self):
        for card in self.hand:
            card.show()
    def redraw(self,deck,redraw_lst):
        for ind in redraw_lst:
            deck.bottom(self.hand[ind])
            self.hand[ind] = deck.draw()
        return self
    def undraw(self,deck,redraw_lst):
        #Search reversed redraw_lst putting card in hand on top of deck and returning bottom of deck to hand in that position
        for ind in reversed(redraw_lst):
            deck.cards.insert(0,self.hand[ind]) #Insert card in hand to top of deck
            self.hand[ind] = deck.draw(-1) #Replace card in this hand position with 
        return self
    #Functions defining what hand we're giving this player
    #Either random or specified hand, with possibly specific values
    def giveRand(self,deck):
        for i in range(5):
            self.hand.append(deck.draw())
        return self
    #Haven't added straight flush
    #So improbable to get normally that there is no expected strategic value to getting rid of
    #This incredibly lucky hand. Royal flush is pointless since you can't do any better.
    def giveSpecific(self,deck,val_lst):
        #Val_lst is 5 specific values, we're gonna ignore flushes for simplicity
        #done = False
        #while not done:
        self.value_lst = []
        for val in val_lst:
            self.value_lst.append(val)
        self.hand = [0 for i in range(5)]
        while 0 in self.hand:
            card=deck.draw()
            if card.value in self.value_lst:
                ind = self.value_lst.index(card.value) #First index location of this value
                #print(ind,card.value,self.value_lst,self.hand)
                self.hand[ind] = card
                self.value_lst[ind] = None
            else:
                deck.bottom(card)
        return self
    #Determine the score quality of the hand and tiebreakers
    def handInfo(self):
        #If we can pass this from previously sorting, do that
        values = [card.value for card in self.hand]
        #Get the unique values and count per value
        counts = bincount(values)
        uniques = nonzero(counts)[0]
        counts = counts[nonzero(counts)]
        #Sort them by counts starting with highest to lowest
        #Then sort count ties by highest to lowest of uniques -- useful for ties
        counts,uniques = zip(*sorted(zip(counts, uniques),reverse=True))
        '''
        4 of a kind
        '''
        if 4 in counts:
            score = 8
        elif 3 in counts:
            '''
            3 of a kind
            '''
            if len(counts) == 3:
                #3 of same value and 2 randos, so not fullhouse
                score = 4
            elif len(counts) == 2:
                '''
                Fullhouse
                '''
                #3 of a kind and a pair --> Fullhouse
                score = 7

        elif 2 in counts:
            #Could be a pair, 2 pair
            '''
            2 pair
            '''
            if len(counts) == 3:
                #2,2,1
                score = 3
            elif len(counts) == 4:
                '''
                1 pair
                '''
                #2,1,1,1
                score = 2
        else:
            #Looking at all singles, check for straight, flush stuff then call it nothing
            '''
            Flush Check
            '''
            flush = True
            suits = []
            for card in self.hand:
                if card.suit not in suits:
                    suits.append(card.suit)
                if len(suits) > 1:
                    flush = False
                    break
            '''
            Straight Check
            '''
            straight = True
            #Due to prior sorting, should be from highest to lowest card
            #As we read along uniques
            #Therefore we can just look for any time where the difference between
            #Current and next value are not 1
            for i in range(4):
                if uniques[i] - uniques[i+1] != 1:
                    straight = False
                    break
            #Now determine type of hand
            if flush and straight:
                if uniques[0] == 14:
                    '''
                    Royal Flush
                    '''
                    score = 10
                    #Technically the highest possible hand and results in legitimate tie
                else:
                    '''
                    Straight Flush
                    '''
                    score = 9
                    #Just check highest card, if equal then tied, if not someone lost/won
            elif flush:
                '''
                Flush
                '''
                score = 6
                ties = uniques
            elif straight:
                '''
                Straight
                '''
                score = 5
                #Again, highest card in straight defines straight
            else:
                '''
                Nothing
                '''
                score = 1
        return score,uniques

In [5]:
def checkWin(player_info,AI_info):
    #Info is list [score,ties]
    #Returning boolean of result in player's perspective
    if player_info[0] > AI_info[0]:
        #Player won
        return True
    elif player_info[0] < AI_info[0]:
        return False
    else:
        #Tied
        if player_info[1] > AI_info[1]:
            return True
        else:
            return False

In [6]:
def AI_Redraw(playerHand):
    score,ties = playerHand.handInfo()
    redraw_lst = []
    if score > 5:
        redraw_lst = []
        #Assume it's not worthwhile to trade in a straight or higher hand
    if score == 4:
        #Three of kind
        #Strategy will be to attempt of fullhouse/ 4 of a kind by trading in both other cards
        #Ties = [Threekind, spare1,spare2]
        for ind,card in enumerate(playerHand.hand):
            if card.value == ties[1] or card.value == ties[2]:
                redraw_lst.append(ind)
    if score == 3:
        #Two Pair
        #Strategy will be to attempt fullhouse
        for ind,card in enumerate(playerHand.hand):
            if card.value == ties[2]:
                redraw_lst.append(ind)
    if score == 2:
        #Pair
        #Keep pair, remove rest
        #Ties = [Pair, Spare1,Spare2,Spare3]
        for ind,card in enumerate(playerHand.hand):
            if card.value == ties[1] or card.value == ties[2] or card.value == ties[3]:
                redraw_lst.append(ind)
    if score == 1:
        #Singles
        #Keep highest, remove rest
        for ind,card in enumerate(playerHand.hand):
            if not card.value == ties[0]:
                redraw_lst.append(ind)
    return redraw_lst

# Step through specific hand, all redraw varieties to find best decision

In [7]:
#All unique hand combinations
card_values = [2,3,4,5,6,7,8,9,10,11,12,13,14]
deck_values = []
for card_value in card_values:
    for i in range(4):
        deck_values.append(card_value)
hand_combs = list(itertools.combinations(deck_values,5))
hand_comb_uniques = unique(array(hand_combs),axis=0)
#Uniques ignoring royal flush, straight flush, flush
#Basically non-flush related ones
global redraw_combs
redraw_combs=[]
for i in range(6):
    test = list(itertools.combinations([0,1,2,3,4],i))
    for j in test:
        redraw_combs.append(j)
        
#Spend a bit more time making them into mutable lists rather than tuples for later usage

In [8]:
print(len(hand_comb_uniques))

6175


# Game Simulations and CSV Writes

In [18]:
def finalSims(startind,stopind,total_games,AI_players,filename):
    for index in range(startind,stopind,+1):
        start = time.time()
        print("Processing Index:",index)
        #Game type value definitions
        #start = time.time()
        games = total_games #150,000 Gives std of around 0.003 which I find acceptable, higher games count decreases but diminishing returns/time
        val_lst = list(hand_comb_uniques[index])
        #Sort val_lst according to poker hand highest to lowest sorting method
        #This can sort the list, maybe don't need to sort playerHands if given this and stick directly with it
        #Get the unique values and count per value
        counts = bincount(val_lst)
        uniques = nonzero(counts)[0]
        counts = counts[nonzero(counts)]
        counts,uniques = zip(*sorted(zip(counts, uniques),reverse=True))
        val_lst = []
        for ind,c in enumerate(counts):
            for i in range(c):
                val_lst.append(uniques[ind])
        
        #Checker to see if we've done this already
        #'''
        df = pd.read_csv(filename, usecols=["Hand List"])
        completed_hands = df["Hand List"].tolist()
        completed_hands = list(unique(completed_hands))
        dummy = []
        for hand in completed_hands:
            dummy.append(ast.literal_eval(hand))
        if val_lst in dummy:
            raise ValueError("We already ran through this hand!!")
        #'''
        
        #Form player hand and determine what the unique ways of redrawing are
        #If we have duplicate values this cuts down runtime a lot and removes duplicate rows
        deck = Deck()
        ph = PlayerHand().giveSpecific(deck,val_lst)
        #ph.sort()
        hand_score,ties = ph.handInfo() #Also grab this for later csv writing
        redraw_remove_value_sets = []
        redraw_unique_combs = []
        
        #Getting just unique redrawing
        for ind, redraw_ind_lst in enumerate(redraw_combs):
            remove = []
            for ind in redraw_ind_lst:
                remove.append(val_lst[ind])
            if remove not in redraw_remove_value_sets:
                redraw_remove_value_sets.append(remove)
                redraw_unique_combs.append(redraw_ind_lst)
        #print(redraw_unique_combs)
        #Simulate games
        wins = [0] * len(redraw_unique_combs)

        for i in range(games):
            deck= Deck()
            playerHand = PlayerHand().giveSpecific(deck,val_lst)
            #Then we haven't added any wins for this redraw type
            #Generate random hands for AI as well as redraw decisions to perform
            AI_Hands = [PlayerHand().giveRand(deck) for i in range(AI_players)]
            AI_redraw_lsts = [AI_Redraw(AI_Hands[i]) for i in range(AI_players)]
            ind = 0
            for redraw_lst in redraw_unique_combs:
                #Redraw for the player
                #Assumed sorted
                playerHand.redraw(deck,redraw_lst)
                #Redrawing for AI players
                for player_ind in range(AI_players):
                    AI_Hands[player_ind].redraw(deck,AI_redraw_lsts[player_ind])
                '''
                AI_redraw_lsts = []
                for player_ind in range(AI_players):
                    AI_redraw_lst = AI_Redraw(AI_Hands[player_ind])
                    AI_redraw_lsts.append(AI_redraw_lst)
                    AI_Hands[player_ind].redraw(deck,AI_redraw_lst)
                '''
                
                #Check if we lost or not
                lost = True
                for i in range(AI_players):
                    if checkWin(playerHand.handInfo(),AI_Hands[i].handInfo()):
                        lost = False
                        break
                #Undo all of the card drawing to reset hands/deck
                for i in range(len(AI_Hands)-1,-1,-1):
                    AI_Hands[i].undraw(deck,AI_redraw_lsts[i])
                playerHand.undraw(deck,redraw_lst)
                if not lost:
                    wins[ind] += 1
                ind += 1

        winrates = [count/games for count in wins]
        stds = [2.576*(((rate)-(rate)**2)/games)**0.5 for rate in winrates] #99% confidence interval
        

        #Sort important lists by winrate
        winrates,redraw_unique_combs,stds = zip(*sorted(zip(winrates,redraw_unique_combs,stds),reverse=True))

        #Info for csv file

        #print("Hand",val_lst)
        #print('')
        keep_inds = []
        keeps = []
        remove_inds = []
        removes = []
        for i in range(len(redraw_unique_combs)):
            keep_ind = []
            keep = []
            remove_ind = []
            remove = []
            for j in range(5):
                if j in redraw_unique_combs[i]:
                    remove.append(val_lst[j])
                    remove_ind.append(j)
                else:
                    keep.append(val_lst[j])
                    keep_ind.append(j)
            '''
            print("Winrate:",winrates[i])
            print("Standard Dev:",stds[i])
            print("Redraw removing:",remove)
            print("Keeping:",keep)
            print("------------------")
            '''
            keep_inds.append(keep_ind)
            keeps.append(keep)
            remove_inds.append(remove_ind)
            removes.append(remove)

        #end = time.time()
        #print("Algorithm Time Elapsed:",end-start)

        end = time.time()
        print("Algorithm Time Elapsed:",end-start)
        #'''
        for i in range(len(redraw_unique_combs)):
            f = open(filename,'a',newline='')
            writer = csv.writer(f)
            row = [val_lst,hand_score,keep_inds[i],keeps[i],remove_inds[i],removes[i],winrates[i],stds[i]]
            writer.writerow(row)
            f.close()
        #'''

In [24]:
#quickChecker(6000,100000,1)#indextotal games, AI_player count, hand values  [12,12,11,11,14]
finalSims(4,1235+1,100000,4,"Hand Winrates 4 AI.csv")#Start index,stop index,total games, AI_player count, optional value list

Processing Index: 4
Algorithm Time Elapsed: 72.94121217727661
Processing Index: 5
Algorithm Time Elapsed: 84.45330381393433
Processing Index: 6
Algorithm Time Elapsed: 84.5876612663269
Processing Index: 7
Algorithm Time Elapsed: 92.73853135108948
Processing Index: 8
Algorithm Time Elapsed: 92.27139830589294
Processing Index: 9
Algorithm Time Elapsed: 100.41243720054626
Processing Index: 10
Algorithm Time Elapsed: 96.98240637779236
Processing Index: 11
Algorithm Time Elapsed: 84.37220001220703
Processing Index: 12
Algorithm Time Elapsed: 101.86582946777344
Processing Index: 13
Algorithm Time Elapsed: 147.88608026504517
Processing Index: 14
Algorithm Time Elapsed: 140.06392979621887
Processing Index: 15
Algorithm Time Elapsed: 142.91494965553284
Processing Index: 16
Algorithm Time Elapsed: 136.25470447540283
Processing Index: 17
Algorithm Time Elapsed: 135.84249234199524
Processing Index: 18
Algorithm Time Elapsed: 129.34091019630432
Processing Index: 19
Algorithm Time Elapsed: 129.47154

ValueError: We already ran through this hand!!